In [96]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args

In [97]:
tickers = ['GOOG', 'AMZN', 'AAPL', 'MSFT', 'META', 'NVDA', 'TSLA']
data = yf.download(tickers, start='2021-01-01', end='2025-04-30')
adj_close = data.xs('Close', axis=1, level='Price')
returns = adj_close.pct_change().dropna()
returns

[*********************100%***********************]  7 of 7 completed


Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,
2021-01-05,0.012364,0.010004,0.007337,0.007548,0.000965,0.022210,0.007317
2021-01-06,-0.033661,-0.024897,-0.003234,-0.028269,-0.025929,-0.058953,0.028390
2021-01-07,0.034123,0.007577,0.029943,0.020622,0.028457,0.057830,0.079447
2021-01-08,0.008631,0.006496,0.011168,-0.004354,0.006093,-0.005040,0.078403
2021-01-11,-0.023249,-0.021519,-0.022405,-0.040102,-0.009698,0.025967,-0.078214
...,...,...,...,...,...,...,...
2025-04-23,0.024332,0.042846,0.024821,0.039958,0.020637,0.038629,0.053662
2025-04-24,0.018426,0.032890,0.023776,0.024756,0.034483,0.036218,0.034976
2025-04-25,0.004367,0.013134,0.014740,0.026484,0.011748,0.043033,0.098031


In [98]:
# train(70%) and test(30%)
split_idx = int(len(returns) * 0.7)
train = returns.iloc[:split_idx]
test = returns.iloc[split_idx:]
train

Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,
2021-01-05,0.012364,0.010004,0.007337,0.007548,0.000965,0.022210,0.007317
2021-01-06,-0.033661,-0.024897,-0.003234,-0.028269,-0.025929,-0.058953,0.028390
2021-01-07,0.034123,0.007577,0.029943,0.020622,0.028457,0.057830,0.079447
2021-01-08,0.008631,0.006496,0.011168,-0.004354,0.006093,-0.005040,0.078403
2021-01-11,-0.023249,-0.021519,-0.022405,-0.040102,-0.009698,0.025967,-0.078214
...,...,...,...,...,...,...,...
2024-01-03,-0.007488,-0.009738,0.005732,-0.005256,-0.000728,-0.012436,-0.040134
2024-01-04,-0.012700,-0.026268,-0.016529,0.007693,-0.007178,0.009019,-0.002181
2024-01-05,-0.004013,0.004634,-0.004709,0.013915,-0.000516,0.022897,-0.001849


In [99]:
test

Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,
2024-01-10,0.005671,0.015591,0.008698,0.036483,0.018574,0.022770,-0.004341
2024-01-11,-0.003223,0.009432,-0.000904,-0.002159,0.004859,0.008684,-0.028725
2024-01-12,0.001778,-0.003609,0.003968,0.013039,0.009984,-0.002043,-0.036661
2024-01-16,-0.012317,-0.009442,-0.001109,-0.018772,0.004634,0.030561,0.004660
2024-01-17,-0.005174,-0.009467,-0.008259,0.002476,-0.002050,-0.005835,-0.019826
...,...,...,...,...,...,...,...
2025-04-23,0.024332,0.042846,0.024821,0.039958,0.020637,0.038629,0.053662
2025-04-24,0.018426,0.032890,0.023776,0.024756,0.034483,0.036218,0.034976
2025-04-25,0.004367,0.013134,0.014740,0.026484,0.011748,0.043033,0.098031


In [100]:
def pca_rotation(returns, n_components=3, lookback=126, rebalance_freq=21, risk_aversion=1.0):
    weights = pd.DataFrame(index=returns.index, columns=returns.columns)
    
    for i in range(lookback, len(returns), rebalance_freq):
        # Rolling window data
        window_returns = returns.iloc[i-lookback:i]
        
        # PCA decompose
        pca = PCA(n_components=n_components)
        pca.fit(window_returns)
        loadings = pca.components_.T  # Asset payload matrix
        
        # Factor return calculation
        factors = pca.transform(window_returns)
        factor_rets = pd.DataFrame(factors, index=window_returns.index).pct_change().dropna()
        
        # factor covariance matrix
        factor_cov = factor_rets.cov()

        # Optimize the factor configuration weights
        def factor_optimization(weights):
            port_ret = weights @ factor_rets.mean()
            port_vol = np.sqrt(weights @ factor_cov @ weights)
            return - (port_ret - risk_aversion * port_vol**2)  # The minus sign is used for minimization
        
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
        bounds = [(-1, 1) for _ in range(n_components)]
        res = minimize(factor_optimization,
                      x0=np.ones(n_components)/n_components,
                      bounds=bounds,
                      constraints=constraints)
        factor_weights = res.x
        
        # Map back to the asset weights
        asset_weights = loadings @ factor_weights
        asset_weights = asset_weights / np.sum(np.abs(asset_weights))  # normalization
        
        # weights allocation
        end_idx = min(i+rebalance_freq, len(returns))
        weights.iloc[i:end_idx] = asset_weights
    
    return weights
        

In [101]:
# default parameters BEFORE optimize
base_params = {
    'n_components': 3,      # Default number of principal components
    'lookback': 126,        # Default half-year window
    'rebalance_freq': 21,   # Monthly portfolio adjustment
    'risk_aversion': 1.0    # Neutral risk aversion
}
base_weights = pca_rotation(test, **base_params)

In [102]:
def optimize_parameters(train_data):
    space = [
        Integer(1, 5, name='n_components'),  # Quantity of principal components
        Integer(63, 252, name='lookback'),   # rolling window
        Integer(5, 63, name='rebalance_freq'),
        Real(0.5, 5.0, name='risk_aversion')
    ]
    
    # Objective function (maximizing the Sharpe ratio)
    @use_named_args(space)
    def objective(**params):
        weights = pca_rotation(train_data, **params)
        port_ret = (weights.shift(1) * train_data).sum(axis=1).dropna()
        sharpe = port_ret.mean() / port_ret.std() * np.sqrt(252)
        return -sharpe
    
    # Bayesian optimization
    res = gp_minimize(objective, space, n_calls=50, random_state=42)
    best_params = {
        'n_components': res.x[0],
        'lookback': res.x[1],
        'rebalance_freq': res.x[2],
        'risk_aversion': res.x[3]
    }
    return best_params


In [103]:
optimized_params = optimize_parameters(train)
print("Optimized parameters:", optimized_params)

Optimized parameters: {'n_components': 3, 'lookback': 69, 'rebalance_freq': 58, 'risk_aversion': 0.5}


In [104]:
print("\nAnalysis of Parameter Importance:")
print(f"- n_components = {optimized_params['n_components']}: "
      f"Uses the first {optimized_params['n_components']} principal components to capture {optimized_params['n_components']} dominant risk factors")
      
print(f"- lookback = {optimized_params['lookback']}: "
      f"{optimized_params['lookback']}-day rolling window balances data recency and stability")

print(f"- rebalance_freq = {optimized_params['rebalance_freq']}: "
      f"Rebalances every {optimized_params['rebalance_freq']} days to balance trading costs with strategy responsiveness")

print(f"- risk_aversion = {optimized_params['risk_aversion']}: "
      f"{'High' if optimized_params['risk_aversion'] > 3 else 'Moderate'} risk preference, "
      f"leading to {'defensive' if optimized_params['risk_aversion'] > 3 else 'balanced'} asset allocation")


Analysis of Parameter Importance:
- n_components = 3: Uses the first 3 principal components to capture 3 dominant risk factors
- lookback = 69: 69-day rolling window balances data recency and stability
- rebalance_freq = 58: Rebalances every 58 days to balance trading costs with strategy responsiveness
- risk_aversion = 0.5: Moderate risk preference, leading to balanced asset allocation


In [105]:
def evaluate_single(weights, single_returns):
    ret = weights.shift(1) * single_returns
    ret = ret.dropna()
    
    metrics = {
        'Strategy Cum Return': (1 + ret).prod() - 1,
        'Annualized Return': (1 + ret).prod()**(252/len(ret)) - 1,
        'Annualized Volatility': ret.std() * np.sqrt(252),
        'Sharpe Ratio': ret.mean() / ret.std() * np.sqrt(252),
        'Max Drawdown': (1 + ret).cummax().sub(1 + ret).div((1 + ret).cummax()).max()
    }
    return pd.Series(metrics)


In [106]:
base_weights = pca_rotation(test, **base_params)

base_results = {}
for ticker in ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']:
    base_results[ticker] = evaluate_single(base_weights[ticker], test[ticker])

base_table = pd.DataFrame(base_results)
print("\nOriginal PCA_rotation Strategy Performance:")
base_table


Original PCA_rotation Strategy Performance:


,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Strategy Cum Return,-0.006957,-0.020700,0.003473,-0.002418,-0.010264,-0.016538,-0.099249
Annualized Return,-0.008802,-0.026141,0.004401,-0.003061,-0.012980,-0.020896,-0.123978
Annualized Volatility,0.031064,0.053088,0.037775,0.072113,0.025980,0.166175,0.184087
Sharpe Ratio,-0.269173,-0.472563,0.134993,-0.006699,-0.489993,-0.043844,-0.626917
Max Drawdown,0.021178,0.033019,0.022206,0.040146,0.016881,0.092032,0.091140


In [107]:
# optimized PCA weights
opt_weights = pca_rotation(test, **optimized_params)

# Calculate independent indicators for each stock
results = {}
for ticker in test.columns:
    results[ticker] = evaluate_single(opt_weights[ticker], test[ticker])

final_results = pd.DataFrame(results)
print("\nPCA_rotation Strategy Performance After Optimized:")
final_results.round(6)


PCA_rotation Strategy Performance After Optimized:


,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA
Strategy Cum Return,0.040983,0.028155,0.036814,0.013026,0.003088,-0.077358,0.254687
Annualized Return,0.040330,0.027709,0.036228,0.012821,0.003040,-0.076196,0.250247
Annualized Volatility,0.018297,0.029612,0.035038,0.042481,0.020006,0.286485,0.236359
Sharpe Ratio,2.170201,0.937814,1.033175,0.321049,0.161658,-0.132543,1.060963
Max Drawdown,0.011832,0.015382,0.015415,0.021665,0.013721,0.148992,0.141046
